#Actual code for ROI analysis

In [ ]:
%matplotlib inline
projecttitle = 'Analogy'
import sys, os
if sys.platform == 'darwin':
    sys.path.append(os.path.join("/Users", "njchiang", "GitHub", "task-fmri-utils"))
else:
    sys.path.append(os.path.join("D:\\", "GitHub", "task-fmri-utils"))

In [2]:
import fmri_core as pa

In [3]:
projectSettings = pa.utils.loadConfig(os.path.join('analogy', 'config', 'project.json'))
analysisSettings = pa.utils.loadConfig(os.path.join('analogy', 'config', 'analyses.json'))

In [4]:
# paths = projectSettings['filepaths']['osxPaths']
paths = projectSettings['filepaths']['winPaths']

ROI Analysis

In [5]:
# for each subject
# mask = lIFGoperc_bin-mask
def runsubject(sub, mask):
    # set image data
    imgFile = os.path.join(paths['root'], 'derivatives', sub, 'betas', pa.utils.formatBIDSName(sub, 'task-analogy', 'betas-pymvpa.nii.gz'))
    mask = pa.utils.loadImg(paths['root'], 
                      'derivatives', sub, 'masks', 
                      mask + '.nii.gz')
    labels = pa.utils.loadLabels(paths['root'], 
                           'derivatives', sub, 'betas', 
                           pa.utils.formatBIDSName(sub, 'task-analogy', 'events-pymvpa.tsv'), 
                           sep='\t', index_col=0)
    # load image
    maskedImg = pa.utils.maskImg(imgFile, mask)
    # clean out timepoints of interest
    conditionSelector = labels['ab'] == 1

    # preprocessing
    from sklearn.preprocessing import StandardScaler
    op = StandardScaler()
    scaledData = pa.utils.opByLabel(maskedImg, labels['chunks'], op)
    # analysis
    from sklearn.svm import SVC
    clf = SVC()
    from sklearn.model_selection import LeaveOneGroupOut
    
    cv = LeaveOneGroupOut().split(scaledData[conditionSelector], labels['trialtype'][conditionSelector], labels['chunks'][conditionSelector])
    result = pa.analysis.roi(scaledData[conditionSelector], labels['abmainrel'][conditionSelector], clf, cv=cv) #, groups=labels['chunks'][conditionSelector])
    # result = pa.unmaskImg(scaledData, mask)
    return result

In [6]:
test = runsubject('sub-01', 'lIFGoperc-bin_mask')

Reading file from: D:\fmri\Analogy\derivatives\sub-01\masks\lIFGoperc-bin_mask.nii.gz


Loading label file from: D:\fmri\Analogy\derivatives\sub-01\betas\sub-01_task-analogy_events-pymvpa.tsv
Masking D:\fmri\Analogy\derivatives\sub-01\betas\sub-01_task-analogy_betas-pymvpa.nii.gz


In [7]:
test

array([ 0.41666667,  0.44444444,  0.41666667,  0.36111111,  0.19444444,
        0.47222222,  0.25      ,  0.33333333])